In [1]:
import sys
sys.path.append('../src')

In [2]:
import torch
from transformer_wrappers.wrappers import CausalLMWrapper
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

In [3]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

In [4]:
COLOURS = [f'C{i}' for i in range(10)]
STYLES = ['solid', 'dotted', 'dashed', 'dashdot']

In [5]:
import os
TOKEN = os.environ['HUGGINGFACE_TOKEN']

In [6]:
# MODEL = 'gpt2-xl'
# MODEL = 'mistralai/Mistral-7B-Instruct-v0.2'  
# MODEL = 'meta-llama/Llama-2-7b-hf'
# MODEL = 'google/gemma-7b'
MODEL = 'google/gemma-2-9b'
MODEL_CONFIGS = {
    'torch_dtype': torch.bfloat16,
    'attn_implementation': 'eager',
    'device_map': DEVICE,
    'token': TOKEN,
}

# TOKENIZER = 'gpt2-xl'
# TOKENIZER = 'mistralai/Mistral-7B-Instruct-v0.2'  
# TOKENIZER = 'meta-llama/Llama-2-7b-hf'
# TOKENIZER = 'google/gemma-7b'
TOKENIZER = 'google/gemma-2-9b'
TOKENIZER_CONFIGS = {'token': TOKEN}

In [7]:
model = CausalLMWrapper.from_pretrained(MODEL, 
                                        model_kwargs=MODEL_CONFIGS, 
                                        tokenizer_name_or_path=TOKENIZER, 
                                        tokenizer_kwargs=TOKENIZER_CONFIGS,
                                        )
model.enable_benchmarking()

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [8]:
input_string = 'The capital of Italy is'

In [9]:
input_encodings = model.tokenizer(input_string, return_tensors='pt').to(DEVICE)

In [10]:
output_wrapper = model(
    **input_encodings,
    return_dict=True,
    output_attentions=True,
    use_cache=True,
    output_hidden_stsates=True,
    return_attention_output=True,  # Self-attention layer output
    return_feed_forward_output=True
)

In [11]:
output = model.generate(**input_encodings, max_new_tokens=32)

AttributeError: 'LMHeadWrapper' object has no attribute 'weight'

In [12]:
model.tokenizer.decode(output['sequences'].squeeze())

NameError: name 'output' is not defined

## Original transformers model

In [30]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it", token=TOKEN)
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b-it",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    token=TOKEN
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

<bos>Write me a poem about Machine Learning.

A tapestry of data, woven tight,
A million threads of knowledge, shining bright.
Machine learning, a weaver's art,
Unravel
